In [3]:
#!/home/was966/micromamba/envs/responder/bin/python
#sbatch --mem 64G -c 4 -t 100:00:00 -p gpu_quad --gres=gpu:rtx8000:1 ./ctct_dense16.py

import sys

sys.path.insert(0, '/home/was966/Research/mims-compass/')
from compass.utils import plot_embed_with_label
from compass import PreTrainer, FineTuner, loadcompass #, get_minmal_epoch
from compass.utils import plot_embed_with_label,plot_performance, score2
from compass.tokenizer import CANCER_CODE

import os
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import random, torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white', font_scale=1.3)
import warnings
warnings.filterwarnings("ignore")

def onehot(S):
    assert type(S) == pd.Series, 'Input type should be pd.Series'
    dfd = pd.get_dummies(S, dummy_na=True)
    nanidx = dfd[dfd[np.nan].astype(bool)].index
    dfd.loc[nanidx, :] = np.nan
    dfd = dfd.drop(columns=[np.nan])*1.
    cols = dfd.sum().sort_values(ascending=False).index.tolist()
    dfd = dfd[cols]
    return dfd


data_path = '../../00_data/'
df_label = pd.read_pickle(os.path.join(data_path, 'ITRP.PATIENT.TABLE'))
df_tpm = pd.read_pickle(os.path.join(data_path, 'ITRP.TPM.TABLE'))
df_tpm.shape, df_label.shape

dfcx = df_label.cancer_type.map(CANCER_CODE).to_frame('cancer_code').join(df_tpm)

df_task = onehot(df_label.response_label)
size = df_label.groupby('sub_cohort').size()
size = size.index + "\n(n = " + size.astype(str) + ")"
cohorts = df_label.groupby('sub_cohort').size().sort_values().index.tolist()
size

sub_cohort
Allen(Ipi)                     Allen(Ipi)\n(n = 39)
Choueiri(Nivo)             Choueiri(Nivo)\n(n = 16)
Gide(Ipi+Nivo)              Gide(Ipi+Nivo)\n(n = 7)
Gide(Ipi+Pembro)         Gide(Ipi+Pembro)\n(n = 25)
Gide(Nivo)                      Gide(Nivo)\n(n = 9)
Gide(Pembro)                 Gide(Pembro)\n(n = 32)
Hugo(Pembro)                 Hugo(Pembro)\n(n = 26)
IMVigor210(Atezo)      IMVigor210(Atezo)\n(n = 298)
IMmotion150(Atezo)    IMmotion150(Atezo)\n(n = 165)
Kim(Pembro)                   Kim(Pembro)\n(n = 45)
Liu(Nivo)                       Liu(Nivo)\n(n = 45)
Liu(Pembro)                   Liu(Pembro)\n(n = 62)
MGH(Uknow)                     MGH(Uknow)\n(n = 34)
Miao(Atezo)                    Miao(Atezo)\n(n = 2)
Miao(Ipi+Nivo)              Miao(Ipi+Nivo)\n(n = 4)
Miao(Nivo)                     Miao(Nivo)\n(n = 11)
Riaz(Nivo)                     Riaz(Nivo)\n(n = 51)
Rose(Atezo)                   Rose(Atezo)\n(n = 35)
Rose(Nivo)                      Rose(Nivo)\n(n = 5)
R

In [6]:
test_pannel = df_label[df_label.sub_cohort == 'SU2CLC1(Pembro)']
train_pannel = df_label[df_label.sub_cohort == 'SU2CLC1(Nivo)']
train_pannel2 = df_label[(df_label.sub_cohort != 'SU2CLC1(Pembro)') & (df_label.cohort == 'SU2CLC1')]

train_pannel.to_csv('./train_pannel_SU2CLC1(Nivo).csv')
test_pannel.to_csv('./test_pannel_SU2CLC1(Pembro).csv')
train_pannel2.to_csv('./train_pannel_~SU2CLC1(Pembro).csv')

In [7]:
len(train_pannel),len(train_pannel2),len(test_pannel)

(49, 69, 33)